# Image analysis using scikit-image & napari

While tools such as FIJI, Imaris etc. can be very easy to use, using Python for image analysis can be very useful:
- Custom scripts allow you to analyse data in exactly the right way (not just what is implemented in a GUI)
- Python allows you to combine image analysis, machine learning and data science (e.g. one script for everything, from raw data to plots).
- Other neuroscience tools are written in Python, and so you can use functionality from these libraries (DeepLabCut, suite2p, BrainGlobe etc.)

## scikit image
[scikit](https://scikit-image.org) is an easy to use Python image processing library. It works well with numpy, it's well documented and easy to install across operating systems.

## napari
[napari](https://napari.org) is a multidimensional image viewer for Python. It allows for high-performance visualisation of data alongside analysis scripts. It's also easy to develop plugins for, so you can share your workflows with colleagues who may not know much Python.

## Load image as numpy array and inspect

## Load the data
We're using [pooch](https://www.fatiando.org/pooch/) to automate downloading the data, and check the hash.

**Run the following cell to download the data in advance**

In [1]:
import tifffile
import pooch
from pathlib import Path
# Use pooch to fetch data if it hasn't already been downloaded
image_url = "https://gin.g-node.org/neuroinformatics/image-analysis-courses/raw/master/misc/DAPI.tif"
image_path = pooch.retrieve(image_url, path=Path.cwd().parent, fname="DAPI.tif", known_hash="6c4a065ef15f84adc5628b26f2b01e3694cea121425b8e349aab0c5b07582468", progressbar=True)

TypeError: retrieve() got an unexpected keyword argument 'progressbar'

In [ ]:
image = tifffile.imread(image_path)
print(image.dtype)
print(image.shape)

## View image in napari

In [ ]:
import napari

# create the viewer and display the image
viewer = napari.view_image(image)

## Use scikit-image to threshold

In [ ]:
# Threshold using otsu's method and view
# Otsu (1979) IEEE Transactions on Systems, Man and Cybernetics. Vol SMC-9, No 1, p62
import skimage

thresholded = skimage.filters.threshold_otsu(image)
binary = image > thresholded
viewer.add_image(binary)

## Try preprocessing and other thresholding algorithms

In [ ]:
# Smooth the image with a gaussian filter and try again
image_smoothed = skimage.filters.gaussian(image, sigma=5)
viewer.add_image(image_smoothed)

thresholded = skimage.filters.threshold_otsu(image_smoothed)
binary_smoothed = image_smoothed > thresholded
viewer.add_image(binary_smoothed)

In [ ]:
thresholded = skimage.filters.threshold_triangle(image_smoothed)
triangle_thresholded = image_smoothed > thresholded
viewer.add_image(triangle_thresholded)

## Clean up the image to improve segmentation

In [ ]:
# Start by removing small objects
min_object_size = 500  # Define a minimum object size to keep (in pixels)
cleaned_image = skimage.morphology.remove_small_objects(
    triangle_thresholded, min_size=min_object_size
)
viewer.add_image(cleaned_image)

In [ ]:
# Run a watershed
# For more details - see https://scikit-image.org/docs/stable/auto_examples/segmentation/plot_watershed.html
import scipy
import numpy as np

# Calculate distance transform
distance = scipy.ndimage.distance_transform_edt(cleaned_image)
viewer.add_image(distance)

# Find local max, and dilate to ensure one peak per cell
coords = skimage.feature.peak_local_max(
    distance, footprint=np.ones((50, 50)), labels=cleaned_image
)
mask = np.zeros(distance.shape, dtype=bool)
mask[tuple(coords.T)] = True
mask = skimage.morphology.binary_dilation(mask)
markers, _ = scipy.ndimage.label(mask)

# Run watershed
labels = skimage.segmentation.watershed(-distance, markers, mask=cleaned_image)
viewer.add_labels(labels)

## Measure cell properties

In [ ]:
props = skimage.measure.regionprops_table(
    labels,
    properties=(
        "area",
        "centroid",
        "area_bbox",
        "orientation",
        "axis_major_length",
        "axis_minor_length",
    ),
)

In [ ]:
import pandas as pd

pd.DataFrame(props)